<a href="https://colab.research.google.com/github/sdx0112/Chinese-Topic-Modelling/blob/main/Google_Colab/Aggregate_Topic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use ChatGLM2 to pick the most similar pre-defined for each given topic and each given title by comparing their embeddings.

In [1]:
!git clone https://github.com/THUDM/ChatGLM2-6B

Cloning into 'ChatGLM2-6B'...
remote: Enumerating objects: 227, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 227 (delta 94), reused 80 (delta 65), pack-reused 96
Receiving objects: 100% (227/227), 7.36 MiB | 19.94 MiB/s, done.
Resolving deltas: 100% (121/121), done.


In [2]:
import os
os.chdir('/content/ChatGLM2-6B')

In [3]:
import pandas as pd
import numpy as np
from collections import Counter

In [4]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.6/416.6 kB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 108.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 127.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.8/288.8 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.2 MB/s eta 0:00:

In [5]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True)
model = AutoModel.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True, device='cuda')
model = model.eval()

A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm2-6b:
- tokenization_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm2-6b:
- configuration_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm2-6b:
- quantization.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm2-6b:
- modeling_chatglm.py
- quantization.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [6]:
import torch
from sklearn.metrics.pairwise import cosine_similarity

## Get embedding from ChatGLM2-6B

In [7]:
def get_glm_embedding(text, device="cuda"):
    inputs = tokenizer([text], return_tensors="pt").to(device)
    resp = model.transformer(**inputs, output_hidden_states=True)
    y = resp.last_hidden_state
    y_mean = torch.mean(y, dim=0, keepdim=True)
    return y_mean.cpu().detach().numpy()

In [8]:
topics_pdf = ['基建', '房地产', '制造业', '科技', '零售消费', '绿色发展', '医药健康', '教育', '交通运输', '出口贸易', '农业农村',
          '文化旅游', '就业', '民生扶贫', '能源', '物资储备', '宏观', '区域发展', '企业发展', '新冠疫情', '地方财政', '税务',
          '国际关系', '金融']

In [9]:
df_para = pd.read_csv('/content/para_topic.csv', encoding='utf_8_sig')

In [10]:
df_title = pd.read_csv('/content/title_topic.csv', encoding='utf_8_sig')

In [11]:
df_para.head()

,Unnamed: 0,ID,TITLE,CONTENT,TOPIC
0,0,m-1608357555-1682084460,会议审议通过了关于强化企业科技创新主体地位的意见关于加强和改进国有经济管理有力支持中国式现代...,中共中央总书记、国家主席、中央军委主席、中央全面深化改革委员会主任习近平4月21日下午主持召...,宏观
1,1,m-1608357555-1682084460,会议审议通过了关于强化企业科技创新主体地位的意见关于加强和改进国有经济管理有力支持中国式现代...,会议审议通过了《关于强化企业科技创新主体地位的意见》、《关于加强和改进国有经济管理有力支持中...,企业发展
2,2,m-1608357555-1682084460,会议审议通过了关于强化企业科技创新主体地位的意见关于加强和改进国有经济管理有力支持中国式现代...,会议指出，强化企业科技创新主体地位，是深化科技体制改革、推动实现高水平科技自立自强的关键举措...,科技
3,3,m-1608357555-1682084460,会议审议通过了关于强化企业科技创新主体地位的意见关于加强和改进国有经济管理有力支持中国式现代...,会议强调，加强和改进国有经济管理，要立足新时代新征程国有经济肩负的使命任务和功能定位，从服务...,企业发展
4,4,m-1608357555-1682084460,会议审议通过了关于强化企业科技创新主体地位的意见关于加强和改进国有经济管理有力支持中国式现代...,会议指出，支持民营经济发展是党中央的一贯方针。促进民营经济发展壮大，要着力优化民营经济发展环...,企业


In [12]:
df_para.shape

(3540, 5)

In [13]:
topics_pred = df_para['TOPIC'].unique().tolist()

In [14]:
titles = df_title['TITLE'].tolist()

In [15]:
topics_title = df_title['TITLE_TOPIC'].unique().tolist()

## Find the most similar topic from the pre-defined list

In [16]:
def find_most_similar_vectors(A, B, threshold = 0.7):
    most_similar_indices = []

    for vector_a in A:
        similarities = cosine_similarity([vector_a], B)
        if np.max(similarities) >= threshold:
          most_similar_index = np.argmax(similarities)
          most_similar_indices.append(most_similar_index)
        else:
          most_similar_indices.append(-1)

    return most_similar_indices

In [17]:
def get_most_similar_topic(topic_a, topic_b, threshold = 0.7):
  embeddings_a = [get_glm_embedding(str(x))[0][0] for x in topic_a]
  embeddings_b = [get_glm_embedding(str(x))[0][0] for x in topic_b]
  similar_idx = find_most_similar_vectors(embeddings_a, embeddings_b, threshold)
  a_b = {}
  for i in range(len(topic_a)):
    a_b[topic_a[i]] = topic_b[similar_idx[i]]
  return a_b

In [18]:
map_pred = get_most_similar_topic(topics_pred, topics_pdf)

In [19]:
map_title = get_most_similar_topic(topics_title, topics_pdf)

In [20]:
map_title_topic = {}
for index, row in df_title.iterrows():
  map_title_topic[row['TITLE']] = map_title[row['TITLE_TOPIC']]

In [21]:
df_para['TOPIC_PRED'] = df_para['TOPIC'].map(map_pred)
df_para['TOPIC_TITLE'] = df_para['TITLE'].map(map_title_topic)

In [22]:
df_para.head()

,Unnamed: 0,ID,TITLE,CONTENT,TOPIC,TOPIC_PRED,TOPIC_TITLE
0,0,m-1608357555-1682084460,会议审议通过了关于强化企业科技创新主体地位的意见关于加强和改进国有经济管理有力支持中国式现代...,中共中央总书记、国家主席、中央军委主席、中央全面深化改革委员会主任习近平4月21日下午主持召...,宏观,宏观,企业发展
1,1,m-1608357555-1682084460,会议审议通过了关于强化企业科技创新主体地位的意见关于加强和改进国有经济管理有力支持中国式现代...,会议审议通过了《关于强化企业科技创新主体地位的意见》、《关于加强和改进国有经济管理有力支持中...,企业发展,企业发展,企业发展
2,2,m-1608357555-1682084460,会议审议通过了关于强化企业科技创新主体地位的意见关于加强和改进国有经济管理有力支持中国式现代...,会议指出，强化企业科技创新主体地位，是深化科技体制改革、推动实现高水平科技自立自强的关键举措...,科技,科技,企业发展
3,3,m-1608357555-1682084460,会议审议通过了关于强化企业科技创新主体地位的意见关于加强和改进国有经济管理有力支持中国式现代...,会议强调，加强和改进国有经济管理，要立足新时代新征程国有经济肩负的使命任务和功能定位，从服务...,企业发展,企业发展,企业发展
4,4,m-1608357555-1682084460,会议审议通过了关于强化企业科技创新主体地位的意见关于加强和改进国有经济管理有力支持中国式现代...,会议指出，支持民营经济发展是党中央的一贯方针。促进民营经济发展壮大，要着力优化民营经济发展环...,企业,房地产,企业发展


In [24]:
df_para[['ID','TITLE','CONTENT','TOPIC_PRED']].to_csv('/content/para_topic_clean.csv', index = False, encoding='utf_8_sig')

In [25]:
df_para[['ID','TITLE','TOPIC_TITLE']].drop_duplicates(subset='ID').to_csv('/content/title_topic_clean.csv', index = False, encoding='utf_8_sig')

## Aggregate to document level

In [ ]:
def group_topics(group):
  topics_group = group['TOPIC_PRED'].tolist()
  counter = Counter(topics_group)
  top = counter.most_common(2)
  topics_all = [group['TOPIC_TITLE'].iloc[0]]
  topics_all.extend([x[0] for x in top])
  topics_all = list(set(topics_all))
  return pd.DataFrame({'ID': group.name,'TOPICS_ALL':[' '.join(topics_all)]})

In [ ]:
result = df_para.groupby('ID').apply(group_topics).reset_index(drop=True)

In [ ]:
result.to_csv('/content/id_topic_2.csv', encoding='utf_8_sig', index = False)